In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%load_ext autoreload
%autoreload 2

## Initialization

In [5]:
import sys
# sys.path.insert(0, '..')

import os
import time
import numpy as np
import argparse
import yaml
from tqdm import tqdm

from dataloaders.nyu_v2_dataloader import NYU_v2
# from dataloaders.cityscapes_dataloader import CityScapes
# from dataloaders.taskonomy_dataloader import Taskonomy
from dev.blockdrop_env_dev import BlockDropEnv_Dev

from utils.util import makedir, print_separator, create_path, print_yaml, should, fix_random_seed, read_yaml_from_input, timestring

# from sklearn.metrics import confusion_matrix
# from models.deeplab_resnet import MTL2, MTL_Instance
# from models.base import Bottleneck, BasicBlock
# from scipy.special import softmax
# import pickle

import torch
from torch import nn
import torch.optim as optim
import torch.optim.lr_scheduler as scheduler
from torch.utils.data import DataLoader
import pprint 
pp = pprint.PrettyPrinter(indent=4)
# from models.base import Bottleneck, BasicBlock

import torch.nn.functional as F
 

In [6]:
from dev.train_dev import eval_dev
from dev.MTL2_Dev import MTL2_Dev
from dev.blockdrop_env_dev import BlockDropEnv_Dev
from dev.deeplab_resnet_dev import Deeplab_ResNet_Backbone_Dev

In [7]:
# input_args = " --config  yamls/adashare/nyu_v2_3task_test.yml --cpu ".split()
input_args = " --config yamls/adashare/nyu_v2_2task.yml --cpu ".split()
# pretrained_path = "/mnt/f/models_adashare/NYU_v2_3Task_pretrain"
print(input_args)
# print(pretrained_path)

['--config', 'yamls/adashare/nyu_v2_2task.yml', '--cpu']


### Create folders and print options 

In [8]:
# ********************************************************************
# ****************** create folders and print options ****************
# ********************************************************************
# read the yaml
print_separator('READ YAML')
opt, gpu_ids, _ = read_yaml_from_input(input_args)
fix_random_seed(opt["seed"][0])
create_path(opt)
# print yaml on the screen
lines = print_yaml(opt)
for line in lines: print(line)
    
# print to file
with open(os.path.join(opt['paths']['log_dir'], opt['exp_name'], 'opt.txt'), 'w+') as f:
    f.writelines(lines)

##################################################
####################READ YAML#####################
##################################################
{'config': 'yamls/adashare/nyu_v2_2task.yml', 'exp_ids': [0], 'gpus': [0], 'cpu': True}
 Create folder ../experiments/logs/nyu_v2_2task
 Create folder ../experiments/results/nyu_v2_2task
 Create folder ../experiments/checkpoints/nyu_v2_2task

exp_name. : nyu_v2_2task
seed. : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
backbone. : ResNet34
original_backbone. : ResNet18
tasks. : ['seg', 'sn']
lambdas. : [1, 20]
tasks_num_class. : [40, 3]
policy_model. : task-specific
paths.
paths.log_dir. : ../experiments/logs/
paths.result_dir. : ../experiments/results
paths.checkpoint_dir. : ../experiments/checkpoints
dataload.
dataload.dataset. : NYU_v2
dataload.dataroot. : /home/kbardool/MLDatasets/nyu_v2
dataload.orig_dataroot. : /data/datasets/nyu_v2
dataload.crop_h. : 321
dataload.crop_w. : 321
policy. : True
init_neg_logits. : None
is_sparse. : T

In [ ]:
opt['dataload']

## Prepare dataloaders

In [ ]:
    # ********************************************************************
    # ******************** Prepare the dataloaders ***********************
    # ********************************************************************
    # load the dataloader
    print_separator('CREATE DATALOADERS')
    
    if opt['dataload']['dataset'] == 'NYU_v2':
        # To warm up
        trainset  = NYU_v2(opt['dataload']['dataroot'], 'train', opt['dataload']['crop_h'], opt['dataload']['crop_w'])
        # To update the network parameters
        trainset1 = NYU_v2(opt['dataload']['dataroot'], 'train1', opt['dataload']['crop_h'], opt['dataload']['crop_w'])
        # To update the policy weights
        trainset2 = NYU_v2(opt['dataload']['dataroot'], 'train2', opt['dataload']['crop_h'], opt['dataload']['crop_w'])
        # To validate
        valset    = NYU_v2(opt['dataload']['dataroot'], 'test')
 
    else:
        raise NotImplementedError('Dataset %s is not implemented' % opt['dataload']['dataset'])
## Prepare dataloaders

In [ ]:
len(valset.__dict__['triples'])

In [ ]:
# IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
# print(IMG_MEAN.shape, IMG_MEAN)
# print(IMG_MEAN[np.newaxis, np.newaxis, :].shape, IMG_MEAN[np.newaxis, np.newaxis, :])
# IMG_MEAN = np.tile(IMG_MEAN[np.newaxis, np.newaxis, :], (480, 640,1))
# print(IMG_MEAN.shape) 
# print(IMG_MEAN[:5,:5,0])
# print(IMG_MEAN[:5,:5,1])
# print(IMG_MEAN[:5,:5,2])


In [ ]:
train_loader  = DataLoader(trainset , batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
train1_loader = DataLoader(trainset1, batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
train2_loader = DataLoader(trainset2, batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
val_loader    = DataLoader(valset   , batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=False)

When Batch Size == 32

In [ ]:
print('size of training set 0 (warm up)       : ', len(trainset))
print('size of training set 1 (network parms) : ', len(trainset1))
print('size of training set 2 (policy weights): ', len(trainset2))
print('size of validation set                 : ', len(valset))
print('                               Total   : ', len(trainset)+len(trainset1)+len(trainset2)+len(valset))

print(f" batch size       : {opt['train']['batch_size']}")
print(f' len train_loader : {len(train_loader)}')
print(f' len train1_loader: {len(train1_loader)}')
print(f' len train2_loader: {len(train2_loader)}')
print(f' len val_loader   : {len(val_loader)}')

##  Create Environment

In [ ]:
opt['train']['weight_iter_alternate'] = opt['train'].get('weight_iter_alternate', len(train1_loader))
opt['train']['alpha_iter_alternate'] = opt['train'].get('alpha_iter_alternate'  , len(train2_loader))

print(opt['train']['weight_iter_alternate'], opt['train']['alpha_iter_alternate'])

In [ ]:
opt['train']['total_iters']   = 30
opt['train']['warm_up_iters'] = 4
opt['train']['print_freq']    = 1
opt['train']['val_freq']      = 2

In [ ]:
print( 
    f"\n backbone                : {opt['backbone']}",
    f"\n paths.log_dir           : {opt['paths']['log_dir']}", 
    f"\n paths.checkpoint_dir    : {opt['paths']['checkpoint_dir']}", 
    f"\n experiment name         : {opt['exp_name']}",
    f"\n tasks_num_class         : {opt['tasks_num_class'],}",
    f"\n init_neg_logits         : {opt['init_neg_logits'],}",
    f"\n device id               : {gpu_ids[0]}",
    f"\n init temp               : {opt['train']['init_temp'],}",
    f"\n decay temp              : {opt['train']['decay_temp']}",
    f"\n fix BN parms            : {opt['fix_BN']}",
    f"\n skip_layer              : {opt['skip_layer']}",
    f"\n train.init_method       : {opt['train']['init_method']}")
print(f" Total iterations        : {opt['train']['total_iters']}")
print(f" Warm-up iterations      : {opt['train']['warm_up_iters']}")
print(f" Print Frequency         : {opt['train']['print_freq']}")
print(f" Validation Frequency    : {opt['train']['val_freq']}")
print(f" Weight iter alternate   : {opt['train']['weight_iter_alternate'] }")
print(f" Alpha  iter alternate   : {opt['train']['alpha_iter_alternate'] }")

In [ ]:
from dev.MTL2_Dev import MTL2_Dev
from dev.blockdrop_env_dev import BlockDropEnv_Dev
from dev.sparsechem_env_dev import SparseChemEnv_Dev

In [ ]:
# del environ
# del BlockDropEnv_Dev
# del MTL2_Dev

In [ ]:
 
# environ = SparseChemEnv_Dev(opt['paths']['log_dir'], 
#                        opt['paths']['checkpoint_dir'], 
#                        opt['exp_name'],
#                        opt['tasks_num_class'], 
#                        opt['init_neg_logits'], 
#                        gpu_ids[0],
#                        opt['train']['init_temp'], 
#                        opt['train']['decay_temp'], 
#                        is_train=True, opt=opt)

In [ ]:
a = torch.Tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12], [13,14,15]])
print(a.shape)
print(a.contiguous().view(-1,1))

### Create environment

In [ ]:
# ********************************************************************
# ********************Create the environment *************************
# ********************************************************************
# create the model and the pretrain model
print_separator('CREATE THE ENVIRONMENT')
environ = BlockDropEnv_Dev(opt['paths']['log_dir'], 
                       opt['paths']['checkpoint_dir'], 
                       opt['exp_name'],
                       opt['tasks_num_class'], 
                       opt['init_neg_logits'], 
                       gpu_ids[0],
                       opt['train']['init_temp'], 
                       opt['train']['decay_temp'], 
                       is_train=True, opt=opt)

In [ ]:
print("environment successfully created")

print(environ.num_tasks)
# hasattr(environ, 'policy1')
print(hasattr(environ, 'policy1'))
# print(environ.networks['mtl-net'])
print(environ.networks['mtl-net'].policys)

##  Training Loop

### Training Loop Preparation

In [ ]:
print(f"which_iter   : {opt['train']['which_iter']}\n"
      f"train_resume :  {opt['train']['resume']}")

print(environ.networks['mtl-net'].layers)

current_iter   = 0
current_iter_w = 0 
current_iter_a = 0

if opt['train']['resume']:
    print(' Resume training')
    current_iter = environ.load(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
else:
    print(' Initiate Training ')


if torch.cuda.is_available():
    environ.cuda(gpu_ids)
else:
    print('cuda not available')
    environ.cpu()
print('\n')

environ.define_optimizer(policy_learning=False)
environ.define_scheduler(policy_learning=False)
# Fix Alpha     - disable gradient flow in the policy network
# Free weights  - allow gradient flow through the weights
environ.fix_alpha()
environ.free_w(opt['fix_BN'])

####  continue training preparation




if opt['dataload']['dataset'] == 'NYU_v2':
    
    if len(opt['tasks_num_class']) == 2:
        
        refer_metrics = {'seg': {'mIoU'         : 0.413, 
                                 'Pixel Acc'    : 0.691},
                          
                         'sn' : {'Angle Mean'   : 15, 
                                 'Angle Median' : 11.5, 
                                 'Angle 11.25'  : 49.2, 
                                 'Angle 22.5'   : 76.7,
                                 'Angle 30'     : 86.8}}
        
    elif len(opt['tasks_num_class']) == 3:
        
        refer_metrics = {'seg': {'mIoU'         : 0.275, 
                                 'Pixel Acc'    : 0.589},
                          
                         'sn' : {'Angle Mean'   : 17.5, 
                                 'Angle Median' : 14.2, 
                                 'Angle 11.25'  : 34.9, 
                                 'Angle 22.5'   : 73.3,
                                 'Angle 30'     : 85.7},
                     
                         'depth':{'abs_err'     : 0.62, 
                                  'rel_err'     : 0.25, 
                                  'sigma_1.25'  : 57.9,
                                  'sigma_1.25^2': 85.8, 
                                  'sigma_1.25^3': 95.7}}
    else:
        raise ValueError('num_class = %d is invalid' % len(opt['tasks_num_class']))
else:
    raise NotImplementedError('Dataset %s is not implemented' % opt['dataload']['dataset'])

flag         = 'update_w'


best_value   = 0 
best_iter    = 0
p_epoch      = 0
best_metrics = None
flag_warmup  = True

if opt['backbone'] == 'ResNet18':
    num_blocks = 8
elif opt['backbone'] in ['ResNet34', 'ResNet50']:
    num_blocks = 18
elif opt['backbone'] == 'ResNet101':
    num_blocks = 33
elif opt['backbone'] == 'WRN':
    num_blocks = 15
else:
    raise ValueError('Backbone %s is invalid' % opt['backbone'])

print(opt['backbone'], 'num_blocks: ', num_blocks)


pp.pprint(refer_metrics)

###  Training Loop 

In [ ]:

# batch_enumerator  = enumerate(train_loader)
# batch_enumerator1 = enumerate(train1_loader)
# batch_enumerator2 = enumerate(train2_loader)

In [ ]:
while current_iter < opt['train']['total_iters']:
    start_time = time.time()

    environ.train()
    current_iter += 1

##---------------------------------------------------------------     
## part one: warm up
##--------------------------------------------------------------- 

    if current_iter < opt['train']['warm_up_iters']:
        print(f"\n{'-'*100}")        
        print(f"** {timestring()} - Training iteration {current_iter} WARMUP iteration: {current_iter}")    
        print(f"{'-'*100}\n")
        
        batch_idx, batch = next(batch_enumerator)
        environ.set_inputs(batch)
        
        environ.optimize(opt['lambdas'], is_policy=False, flag='update_w')
        
        ## If we've exhausted the dataload, reset to beginning
        if batch_idx == len(train_loader) - 1:
            batch_enumerator = enumerate(train_loader)

        if should(current_iter, opt['train']['print_freq']):
            environ.print_loss(current_iter, start_time)
            environ.resize_results()

        # validation
        if should(current_iter, opt['train']['val_freq']):
            print(f"**  {timestring()}  START VALIDATION iteration: {current_iter} ")    
            
            environ.eval()     # set to evaluation mode (train = False)
            num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
            val_metrics = training_validation(environ, 
                                              val_loader, 
                                              opt['tasks'], 
                                              policy=False, 
                                              num_train_layers=None, 
                                              num_seg_cls=num_seg_class,
                                              eval_iter = 3)
            
            environ.print_loss(current_iter, start_time, val_metrics, title='validation')
            environ.save_checkpoint('latest', current_iter)
            
            print(f"** {timestring()} - END VALIDATION iteration:  {current_iter} ")                
            environ.train()    # set to training mode (train = True)

##------------------------------------------------------------------------------------------
## part two:   current_iter >= warm_up_iters
##            - set flag_warmup switch to false
##            - first iteration here - save warmup model to checkpoint 
##                                   - turn off gradient flow through policy network    
##------------------------------------------------------------------------------------------


    else:
        print_heading(f"** {timestring()} - Training iteration {current_iter}  flag: {flag} ")    

        if flag_warmup:
            environ.define_optimizer(policy_learning=True)
            environ.define_scheduler(policy_learning=True)
            flag_warmup = False

        if current_iter == opt['train']['warm_up_iters']:
            environ.save_checkpoint('warmup', current_iter)
            environ.fix_alpha()

        #-----------------------------------------
        # Train & Update the network weights
        #-----------------------------------------
        if flag == 'update_w':
            
            current_iter_w += 1
            batch_idx_w, batch = next(batch_enumerator1,1)
            environ.set_inputs(batch)

            ##----------------------------------------------------------------------
            ## Set number of layers to train based on cirriculum_speed 
            ## and p_epoch (number of epochs of policy training)
            ## When curriculum_speed == 3, a num_train_layers is incremented 
            ## after completion of every 3 policy training epochs
            ##----------------------------------------------------------------------
            if opt['is_curriculum']:
                num_train_layers = p_epoch // opt['curriculum_speed'] + 1
            else:
                num_train_layers = None
                
            print(f"++ CALL ENVIRON.OPTIMIZE() flag: {flag} is_policy: {opt['policy']} p_epoch: {p_epoch}  num_train_layers: {num_train_layers}\n") 
            
            environ.optimize(opt['lambdas'], 
                             is_policy=opt['policy'], 
                             flag=flag, 
                             num_train_layers=num_train_layers,
                             hard_sampling=opt['train']['hard_sampling'])
 
            ## if (current_iter % print_freq) == 0
            if should(current_iter, opt['train']['print_freq']):
                environ.print_loss(current_iter, start_time)
                environ.resize_results()

            ## when current_iter_w == len(train1_loader) switch to policy training

            print(f"++ current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w} current_iter_a: {current_iter_a} ['weight_iter_alternate']:"\
                  f" {opt['train']['weight_iter_alternate']}")            
            
            
            ## if (current_iter_w % weight_iter_alternate) == 0 
            
            if should(current_iter_w, opt['train']['weight_iter_alternate']):
                print(f"++ Switch to update_alpha")
                flag = 'update_alpha'
                environ.fix_w()
                environ.free_alpha()
                
                #-------------------------------------------------------
                # START validation process
                #-------------------------------------------------------                
                environ.eval()
                print('++ Weight Training Validation ...')
                
                num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
                
                val_metrics = training_validation(environ, 
                                                  val_loader, 
                                                  opt['tasks'], 
                                                  policy=opt['policy'],
                                                  num_train_layers=num_train_layers, 
                                                  hard_sampling=opt['train']['hard_sampling'],
                                                  num_seg_cls=num_seg_class)
                
                environ.print_loss(current_iter, start_time, val_metrics)
                environ.save_checkpoint('latest', current_iter)
                
                print(f" current iter: {current_iter}       warm_up_iters: {opt['train']['warm_up_iters']} \n"
                      f" num_blocks  : {num_blocks}         cirriculum_speed: {opt['curriculum_speed']}\n"
                      f" opt['train']['weight_iter_alternate'] : {opt['train']['weight_iter_alternate']}\n"
                      f" opt['train']['alpha_iter_alternate']  : {opt['train']['alpha_iter_alternate']}\n"
                      f" {current_iter - opt['train']['warm_up_iters']}  ??"
                      f" {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])}")
                                             
                ## if the training iteration is larger than a certain amount 
                ## (num_blocks (8 for ResNet))* (curriculum_speed(3)) * (weight training iterations + policy training iterations (19+4))
                ## check metrics for improvement, and issue a checkpoint if necessary
                
                if current_iter - opt['train']['warm_up_iters'] >= num_blocks * opt['curriculum_speed'] * \
                        (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']):
                    new_value = 0

                    for k in refer_metrics.keys():
                        if k in val_metrics.keys():
                            for kk in val_metrics[k].keys():
                                if not kk in refer_metrics[k].keys():
                                    continue
                                if (k == 'sn' and kk in ['Angle Mean', 'Angle Median']) or 
                                   (k == 'depth' and not kk.startswith('sigma')) or 
                                   (kk == 'err'):
                                    value = refer_metrics[k][kk] / val_metrics[k][kk]
                                else:
                                    value = val_metrics[k][kk] / refer_metrics[k][kk]
                                ## matching keys count
                                ## matching_keys_cnt = len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
                                value = value / len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
                                new_value += value
                    
                    print('Best Value %.4f  New value: %.4f' % new_value)

                    if new_value > best_value:
                        print('Previous best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)
                        best_value = new_value
                        best_metrics = val_metrics
                        best_iter = current_iter
                        environ.save_checkpoint('best', current_iter)
                        print('New      best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)                    

                environ.train()
                #-------------------------------------------------------
                # END validation process
                #-------------------------------------------------------       
                
            print(f"++ Current loader (train1) index: {batch_idx_w}   len(train1_loader) = {len(train1_loader)} ")
            
            if batch_idx_w == len(train1_loader):
                batch_enumerator1 = enumerate(train1_loader,1)
                
        #---------------------------------------------------------------------------
        # Train & update the policy network
        #---------------------------------------------------------------------------
        elif flag == 'update_alpha':
            
            current_iter_a += 1
            batch_idx_a, batch = next(batch_enumerator2, 1)
            environ.set_inputs(batch)
            
            if opt['is_curriculum']:
                num_train_layers = p_epoch // opt['curriculum_speed'] + 1
            else:
                num_train_layers = None
                
            print(f"++ CALL ENVIRON.OPTIMIZE() flag: {flag} is_policy: {opt['policy']} p_epoch: {p_epoch}  num_train_layers: {num_train_layers}\n") 
            
            environ.optimize(opt['lambdas'], 
                             is_policy=opt['policy'], 
                             flag=flag, 
                             num_train_layers=num_train_layers,
                             hard_sampling=opt['train']['hard_sampling'])

            if should(current_iter, opt['train']['print_freq']):
                environ.print_loss(current_iter, start_time)
                environ.resize_results()
                # environ.visual_policy(current_iter)
            
            ## when current_iter_a == len(train2_loader) switch to policy training
            print(f"** current_iter_a: {current_iter_a} ['alpha_iter_alternate']: {opt['train']['alpha_iter_alternate']}",
                  f"   batch_idx_a {batch_ix_a}  Current iter w: {current_iter_w} ")            
           
            ## if (current_iter_a % alpha_iter_alternate) == 0 
            
            if should(current_iter_a, opt['train']['alpha_iter_alternate']):
                print(f"** Switch training to update_weight")                
 
                flag = 'update_w'
  
                environ.fix_alpha()
                environ.free_w(opt['fix_BN'])
                environ.decay_temperature()
                
                # print the distribution
                dists = environ.get_policy_prob()
                print(np.concatenate(dists, axis=-1))
                p_epoch += 1
                print(f"** p_epoch incremented: {p_epoch}")

            
            print(f"++ Current loader (train2 index:: {batch_idx_a} len(train2_loader) = {len(train2_loader)}")                
            if batch_idx_a == len(train2_loader):
                batch_enumerator2 = enumerate(train2_loader,1)

        else:
            raise ValueError('flag %s is not recognized' % flag)
            
            


In [ ]:
( 43 // 3 )+ 1

### Display parameters of the network. 

`taskn_logits` are architecture parms
others are network parms

In [ ]:

print(environ.networks['mtl-net'].layers)
print(environ.networks['mtl-net'].backbone.layer_config)
for m in environ.networks['mtl-net'].backbone.modules():
#     print(type(m))
    print('==> ',type(m),'\n',m)
    if isinstance(m, nn.Conv2d):
        print(m.kernel_size, m.out_channels)

In [ ]:
planes = 128
keep_channels = (planes * np.cumsum([0, 0.25, 0.25, 0.5])).astype(np.int32)
print(keep_channels)
keep_masks = []
for kc in keep_channels:
    mask = np.zeros([1, planes, 1, 1])
    print(f' keep channels: {kc:2d}  mask: {mask.shape} - set mask[:, :{kc:2d}] =1')
    mask[:, :kc] = 1
    keep_masks.append(mask)
    
for i in keep_masks:
    print(type(i), i.shape, i.sum())
    
keep_masks_py = torch.from_numpy(np.concatenate(keep_masks)).float()

print(keep_masks_py)

In [ ]:
dilation = 1 
kernel_size = np.asarray((3, 3))
upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
print(upsampled_kernel_size)

In [ ]:
# print(environ.get_arch_parameters())
for i,j in environ.networks['mtl-net'].named_parameters():
    print(f"name: {i}    shape: {j.shape}")

## Classes

### Call MTL2

In [ ]:
block = BasicBlock
layers = [2, 2, 2, 2]
num_classes_tasks =  opt['tasks_num_class']
init_method =  opt['train']['init_method']
init_neg_logits = opt['init_neg_logits']
skip_layer = opt['skip_layer']

In [ ]:
mtl2 = MTL2_Dev(block, layers, num_classes_tasks, init_method, init_neg_logits, skip_layer)
